## MNS Practical 7:
Leonard Hollander

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings 

#warnings.filterwarnings('ignore')

## Exercise 1: Hodging-Huxley model of action potential generation

In [2]:
def alpha_n (V):
    
    return .01 * (V + 55) / (1 - np.exp(-.1 * (V + 55)))

def beta_n (V):

    return .125 * np.exp(-.0125 * (V + 65))

def alpha_m (V):
    
    return (.1 * (V + 40)) / (1 - np.exp(-.1 * (V + 40)))

def beta_m (V):

    return 4 * np.exp(-.0556 * (V + 65))

def alpha_h (V):
    
    return .07 * np.exp(-.05 * (V + 65))

def beta_h (V):

    return 1 / (1 + np.exp(-.1 * (V + 35)))


In [3]:
def euler(f_func, x_0, t_max, dt):
    
    x = x_0
    nsteps = int(t_max / dt)
    xs = np.zeros(nsteps)
    xs[0] = x_0
    i = 1
    t = 0
    
    while i < nsteps:
        
        x = x + f_func(x,t)*dt
        xs[i] = x
        t += dt
        i += 1
    
    return xs


In [4]:
#Defining initial conditions:

m_0 = .0529
h_0 = .5961
n_0 = .3177

In [5]:
#Potassium constants:

E_k = -77 #mV
gk_max = 36 #nS

#Sodium constants:

E_Na = 50
gNa_max = 120

#Solving the system of ODEs:

"""

dm = lambda m, t, V: alpha_m(V) * (1 - m) - beta_m(V) * m
m = lambda V: euler(lambda m, t: dm(m,t, V), m_0, 25, .01)

dh = lambda h, t, V: alpha_h(V) * (1 - h) - beta_h(V) * h
h = lambda V: euler(lambda h, t: dh(h,t, V), h_0, 25, .01)

dn = lambda n, t, V: alpha_n(V) * (1 - n) - beta_n(V) * n
n = lambda V: euler(lambda n, t: dn(n,t, V), n_0, 25, .01)

"""

#Current equations:

I_k = lambda n, V: gk_max * np.power(n(V),4) * (V - E_k)
I_Na = lambda m, h, V: gNa_max * np.power(m(V),3) * h(V) * (V - E_Na)

In [6]:
#Leak current:

g_leak = .3
E_leak = -54.387

#Leak current equation:

I_leak = lambda V: g_leak * (V - E_leak)

$\textbf{Part[a]: Simulating the Hodgkin-Huxley model }$

In [7]:
from scipy.integrate import odeint

In [8]:
#Defining some Hodgking-Huxley constants:

def func (x,t):
    
    n = x[0]
    h = x[1]
    m = x[2]
    V = x[3]
    
    #Define temporal dynamics of channel gates:
    
    dndt = alpha_n(V) * (1 - n) - beta_n(V) * n
    dhdt = alpha_h(V) * (1 - h) - beta_h(V) * h
    dmdt = alpha_m(V) * (1 - m) - beta_m(V) * m
    
    #Get the voltage:
    
    dVdt = -I_Na(m,h, V) -I_k(n, V) - I_leak(V) + I_e
    
    return [dndt, dhdt, dmdt, dVdt]

print(func([n_0, h_0, m_0, -60]))
    
    
    





TypeError: func() missing 1 required positional argument: 't'

In [ ]:
#Addendum: Calculate gating dynamics for analytical sheet 9:

def m_func(t):
    
    return .96 * (1 - np.exp(-t/ .25))

np.vectorize(m_func)

def h_func(t):
    
    return .6 * np.exp(-t/.84)

np.vectorize(h_func)

def n_func(t):
    
    return .89 - .38 * np.exp(-t/1.7)

np.vectorize(n_func)

gNA_new = 70.7
gk_new = 24.3

timecourse = np.arange(0,10,.1)

plt.plot(timecourse, n_func(timecourse), label = "n")
plt.plot(timecourse, m_func(timecourse), label = "m")
plt.plot(timecourse, h_func(timecourse), label = "h")
plt.legend()



In [ ]:
Æ¿¿/#Part(b):

def gNA(t):
    
    return 70.7 * np.power(m_func(t),3) * h_func(t)

np.vectorize(gNA)

def gK(t):
    
    return 24.3 * np.power(n_func(t),4)

np.vectorize(gK)

plt.figure()

plt.plot(timecourse, gNA(timecourse), label = "gNA")
plt.plot(timecourse, gK(timecourse), label = "gK")
plt.legend()

In [ ]:
print(np.power(.89,4) * 24.3)
np.max(gNA(timecourse)), np.argmax(gNA(timecourse))               

In [ ]:
rt f